# 3 Assignment RNN / LSTM DMML
Thomas Delissen - November 2024

This assignment is based on the Blog from Andrej Karpathy: https://karpathy.github.io/2015/05/21/rnn-effectiveness/ 

That Website might give you some useful hints. Note that you cannot copy the code from that Github, since the assignment you will have to do is to create an LSTM using Keras instead of implementing a manual RNN. 

The goal will be to create an LSTM using Keras that can generate long sequences of characters, hopefully resembling the input text. So this will be a NLP Sequence to Sequence Model that can generate single characters. 

This exercise uses the file tinyshakespeare_karpathy.txt . 

In [1]:
# The following libraries might come in handy
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical

# data I/O - code copied from Andrej Karphaty github
data = open('tinyshakespeare_karpathy.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

print("Data size: " + str(data_size))
print("Vocab size: " + str(vocab_size))
print("Available Characters: " + str(chars))

Data size: 1115394
Vocab size: 65
Available Characters: ['k', 'C', 'U', 'L', 'x', 'D', 'e', 'w', 'm', 'M', 'V', 'p', 'Y', '&', 'Q', 'S', '3', 'J', 'u', 'H', 'X', 'o', ',', 'q', 'I', 'A', 'B', '!', 'W', ';', 'f', 's', 'c', 'n', '$', 'Z', 'l', 'T', 'F', 'O', 'v', 'R', 't', '?', 'j', 'K', '-', 'r', 'E', 'a', 'g', 'h', 'y', "'", '\n', 'i', 'b', 'G', '.', 'N', 'z', ':', 'P', ' ', 'd']


Dictionaries char_to_ix and ix_to_char can be used to convert the character to a number and vice versa. 

In [2]:
print(char_to_ix["E"])
print(ix_to_char[2])

48
U


So, we have read in the data in your notebook. Your assignment is the following: 

1. Prepare the data for training an LSTM

We will take an input sequence lenght of 40 characters. Proposed solution: Create an X and a y variable of type numpy array, where each row in X should be 40 characters, encoded as numbers, and each row in y should contain the next character, also encoded as a number. Once the numpy arrays have been created, use the to_categorical function from keras to one-hot-encode the arrays. We do not need to split off a test set, since we will not evaluate the LSTM, just use it to generate text. 

2. Design an LSTM for the training data you just created, compile it and train it. 

You can decide yourself on the architecture, can be simple or complex, as you like. Proposal: Your final layer will most likely be a multiclass classification of size vocab_size, so you should use the appropriate activation function for that. 

3. Create a function (def) that can iteratively call your RNN / LSTM

The function should expect as an input an initial (40) character string, so that you can generate the first next character. A second parameter of the function should be the number of tokens you like to generate. Inside the function there should be a for loop that iteratively calls your model. Append your generated tokens to the initial string and feed your string into your LSTM iteratively to generate the next token. Output the completely generated text. 

4. Demonstrate your function

Call your function with a string you defined yourself, and let it generate 200 characters. Hopefully, it resembles syntactically correct English. 

For Reference, here is a text that my model generated: 

____________________________
First Citizen:
Before we proceed any further with the cours.

PETRUCHIO:
What shall be the death the strength the strength.

KING RICHARD III:
The shall be the death the strength the strength.

KING RICHARD III:
The shall be the death the s

_____________________________

That is it! You are allowed to use parts from the Github I recommended, and are also allowed to use ChatGPT. But, very important: You must be able to explain your code, so you can only use stuff from ChatGPT that you understand. The model must be created using Keras and Tensorflow. It is allowed to deviate from the detailed instructions from above, as long as you can explain why you did that and as long as your model is able to generate text. In case you do not manage to complete the entire assignment (perhaps only part 1 and 2), please still submit your work, you will get points for it.

#### 1) Prepare the data for training an LSTM
We will take an input sequence lenght of 40 characters. Proposed solution: Create an X and a y variable of type numpy array, where each row in X should be 40 characters, encoded as numbers, and each row in y should contain the next character, also encoded as a number. Once the numpy arrays have been created, use the to_categorical function from keras to one-hot-encode the arrays. We do not need to split off a test set, since we will not evaluate the LSTM, just use it to generate text.

In [5]:
X, y = [], []

seq_length = 40
for char in range(seq_length, len(data) - 1):
    row = data[char - seq_length: char]
    predict_char = data[char]
    predict_char_encoded = char_to_ix[predict_char]
    row_encoded = [char_to_ix[c] for c in row]

    X.append(row_encoded)
    y.append(predict_char_encoded)

X = np.array(X)
y = np.array(y)
print(X.shape)

y = to_categorical(y, num_classes=vocab_size)



(1115353, 40)


#### 2) Design an LSTM for the training data you just created, compile it and train it.

You can decide yourself on the architecture, can be simple or complex, as you like. Proposal: Your final layer will most likely be a multiclass classification of size vocab_size, so you should use the appropriate activation function for that.

In [6]:
# Design an LSTM for the training data you just created, compile it and train it. You can decide yourself on the architecture, can be simple or complex, as you like. Proposal: Your final layer will most likely be a multiclass classification of size vocab_size, so you should use the appropriate activation function for that.

X = np.reshape(X, (X.shape[0], X.shape[1], 1))

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))

# we use categorical_crossentropy as loss function, since we are dealing with a multiclass classification problem and not a regression problem
model.compile(loss='categorical_crossentropy', optimizer='adam')

D:\Uni\5. Semester\Data Mining\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
 4505/27884 ━━━━━━━━━━━━━━━━━━━━ 22:54 59ms/step - loss: 2.5479